In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import shap

c:\Users\Firew Ayele\Desktop\kifiya\AlphaCare-_nsurance_Solutions\.venv_insurance\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load and prepare the data

In [3]:
data = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|', low_memory=False)